In [1]:
import os
import re
import open3d as o3d
import numpy as np
import h5py
import random
import time
import torch
from torch.nn import Softmax

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Print grid in readable format
def pretty_print(grid):
    for row in grid:
        print(row)

#from path of description file get the coupling grid, model data and 
#remove models with error
def process_file(path):
    with open(path,'r') as f:
        #don't consider first two lines
        f.readline()           #segmenti x x x
        f.readline()           #rotazioni x x x

        line=f.readline()      #numero pezzi x  

        #get number of fragments
        number=int(re.findall("\d+",line)[0])

        #coupling matrix
        grid=[]
        for i in range(number):
            line=f.readline()
            ret=re.findall('-1|0|1',line)
            grid.append(list(map(int,ret)))

        #model data
        model_names=[]
        non_valid_indeces=[]
        for m in range(number):
            f.readline()        #blank line
            name=f.readline()   #model name
            mesh=f.readline()   #mesh n
            f.readline()        #external n
            f.readline()        #internal n

            #if mesh=0 I don't consider the element
            mesh_n=int(mesh.rstrip().split(" ")[1]) 
            if mesh_n!=0: 
                #try create file name
                file_name= name.rstrip().replace(".","_")
                model_names.append(file_name)          
            else:
                #saving indices to remove later
                non_valid_indeces.append(m)

        #removing elements from grid
        #sorted in reverse to avoid wrong index
        for index in sorted(non_valid_indeces, reverse=True):
            #remove row
            del grid[index]
            #remove columns
            for row in grid:
                del row[index]
        
        return grid,model_names


In [3]:
#get a set of fragments (i.e. a subfolder)
def get_set(folder,verbose=True):
    folder_path=os.path.join(main_path,folder)
    models_file=[]

    #files are either text files or models
    for file in os.listdir(folder_path):
        if '.txt' in file:
            description_file=file
        elif '.stl' in file:
            #not used, they are not in the same order of the file
            models_file.append(file)

    #parsing description file
    full_description_file=os.path.join(folder_path,description_file)
    grid,model_names=process_file(full_description_file)

    #get path of models of current set
    model_prefix=folder.replace("generatedTest_","")
    complete_models_path=[]
    for i in range(len(model_names)):
        #e.g. 2021_11_29_10_00_40_Cube_001.stl
        correct_model_name=f"{model_prefix}_{model_names[i]}.stl"
        #saving only names, could be useful but now not used
        model_names[i]=correct_model_name

        complete_models_path.append(os.path.join(folder_path,correct_model_name))

    #create set and put into list of sets
    set={"models":complete_models_path,"grid":grid}
    
    if verbose:
        print("A set: \n")
        print("Models: ",set["models"])
        print("Grid: ")
        pretty_print(set["grid"])

    return set

In [4]:
#Create all the possible pairs of fragments
#duplicates are not considered, e.g.: (i,j) - (j,i)
def create_pairs(num):
    lista = []
    for i in range(num):
        for j in range(i+1, num):
            lista.append((i, j))

    return lista

print(create_pairs(5))



[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [5]:
def count_dataset_pairs(main_path,max_elements=-1,alpha=1):
    num_total_pairs=0
    num_models=0
    num_fragments=0
    pairs_at_folder=[]  # save number of total pairs at the end of each folder, needed to split in train-test
    #get total number of pairs we consider:
    
    for folder in os.listdir(main_path):
        #check only folders, not files
        if '.' not in folder:
            fragment_set=get_set(folder,verbose=False)  #set of fragments of one model
            num_models+=1
            num_fragments+=len(fragment_set["models"])
            #get total number of adjacent pairs of fragments
            #i.e. number of 1 in the grid
            grid = np.array(fragment_set["grid"])
            unique, counts = np.unique(grid, return_counts=True)
            dic = dict(zip(unique, counts))
            #divide by two because we consider half of the pairs, not both of (i,j),(j,i)
            num_zeros = int(dic[0]/2)
            num_ones = int(dic[1]/2)

            #consider only alpha*N not adjacent pairs
            max_not_adjacent_pairs=alpha*num_ones
            estimated_num_pairs=int(min(max_not_adjacent_pairs,num_zeros)+num_ones)
            num_total_pairs+=estimated_num_pairs
            pairs_at_folder.append(num_total_pairs)

    if max_elements>0:
        num_total_pairs=min(max_elements,num_total_pairs)
    print("dataset will contain:", num_total_pairs," total pairs")
    print("dataset has ",num_models," models and ",num_fragments," fragments")
    return num_total_pairs,pairs_at_folder
   


In [6]:
#from the mesh sample the pointcloud and translate to center
def get_pointcloud(mesh,num_points):
    pointcloud=mesh.sample_points_poisson_disk(num_points)

    center=pointcloud.get_center()
    pointcloud=pointcloud.translate(center*-1)

    return pointcloud


In [7]:
#alternative approach to getting normals, not used
#for each point of the point cloud get the normals of the nearest vertices in the original mesh
def get_nearest_normal(point,mesh):
    min_index=None
    min_dist=float("inf")
    for i in range(len(mesh.vertices)):
        v=mesh.vertices[i]
        squared_dist = np.sum((point-v)**2, axis=0)
        dist = np.sqrt(squared_dist)
        if dist<min_dist:
            min_dist=dist
            min_index=i
    return mesh.vertex_normals[i]

def get_nearest_normals(pointcloud,mesh):
    normals=[]
    for p in pointcloud.points:
        normals.append(get_nearest_normal(p,mesh))
    return normals

In [8]:
def create_dataset(main_path,num_points,max_elements=-1,alpha=1):
    
    num_elements=0
    num_pairs=0             # number of pairs we put in dataset (train+test)
    num_train_pairs=0       # number of pairs in train
    num_test_pairs=0        # number of pairs in test
    num_val_pairs=0
    num_total_pairs=0       # total number of pairs, including those we don't put in dataset (~1.5 Millions with the smaller dataset)
    folder_index = 0
    tot_num_adjacent=0

    sets=[]

    
    prob_test=0.15
    prob_val=0.15
    prob_train=1-prob_test-prob_val

    print("Creating dataset...")

    dataset_total_pairs,pairs_at_folder=count_dataset_pairs(main_path,max_elements,alpha)

    dataset_file=None
    dataset_file_name=f"dataset_{dataset_total_pairs}pairs_{num_points}points_center_random_normals_chunk_alpha{alpha}.hdf5"

    with open(".gitignore","a") as f:
        f.write(f"{dataset_file_name}\n")

    try: 
        dataset_file.close()
        os.remove(dataset_file_name)
    except:
        print("file dataset not found, creating")

    chunks_size=100
    dataset_file=h5py.File(dataset_file_name, 'w')

    train_total_pairs=int(dataset_total_pairs*(prob_train))
    train_data=dataset_file.create_dataset("train_data", (train_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    train_normals=dataset_file.create_dataset("train_normals", (train_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    train_labels=dataset_file.create_dataset("train_labels", (train_total_pairs,),dtype='i',chunks=(chunks_size,))
    
    test_total_pairs=int(dataset_total_pairs*prob_test)
    test_data=dataset_file.create_dataset("test_data", (test_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    test_normals=dataset_file.create_dataset("test_normals", (test_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    test_labels=dataset_file.create_dataset("test_labels", (test_total_pairs,),dtype='i',chunks=(chunks_size,))

    val_total_pairs=int(dataset_total_pairs*prob_test)
    val_data=dataset_file.create_dataset("val_data", (val_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    val_normals=dataset_file.create_dataset("val_normals", (val_total_pairs,2,num_points,3),chunks=(chunks_size,2,num_points,3))
    val_labels=dataset_file.create_dataset("val_labels", (val_total_pairs,),dtype='i',chunks=(chunks_size,))
    

    total_dataset_start_time=time.time()

    #indexes for saving data on file
    #start from -1 because dataset start at index 0
    train_index=-1
    test_index=-1
    val_index=-1

    print(f"Each point cloud is sampled with {num_points} points\n\n")
    for folder in os.listdir(main_path):
        #check only folders, not files
        if '.' not in folder:
            set_start_time=time.time()
            print(f"\n\nStarting set {folder_index}: folder - {folder}")  

            set_train_data=[]
            set_train_labels=[]
            set_train_normals=[]

            set_test_data=[]
            set_test_labels=[]
            set_test_normals=[]

            set_val_data=[]
            set_val_labels=[]
            set_val_normals=[]

            #GET SET INFORMATIONS
            fragment_set=get_set(folder,verbose=False)  #set of fragments of one model
            num_elements+=len(fragment_set["models"])
            sets.append(fragment_set)
            
            #save pointcloud of meshes
            set_pointcloud=[]
            set_normals=[]

            for path in fragment_set["models"]:
                mesh=o3d.io.read_triangle_mesh(path)
                pointcloud=get_pointcloud(mesh,num_points)
                #normals=get_nearest_normals(pointcloud,mesh)

                set_pointcloud.append(pointcloud)
                #set_normals.append(normals)
                

            #get indeces of pairs
            pairs=create_pairs(len(fragment_set["models"]))

            #shuffle to get random pairs not in order
            random.shuffle(pairs)

            #get total number of adjacent pairs of fragments
            #i.e. number of 1 in the grid
            grid = np.array(fragment_set["grid"])
            unique, counts = np.unique(grid, return_counts=True)
            dic = dict(zip(unique, counts))
            #divide by two because we consider half of the pairs, not both of (i,j),(j,i)
            num_zeros = int(dic[0]/2)
            num_ones = int(dic[1]/2)

            #considerare solo a*N coppie non adiacenti
            max_not_adjacent_pairs=alpha*num_ones

            estimated_num_pairs=int(min(max_not_adjacent_pairs,num_zeros)+num_ones)

            print("Set stats: ")
            print(f"  --number of fragments: {len(fragment_set['models'])}")
            print(f"  --total adjacent pairs: {num_ones}; total not adjacent pairs: {num_zeros}")
            print(f"  --dataset for set will contain: {estimated_num_pairs} pairs")
            print(f"     --> adj: {num_ones}    n-adj: {int(min(max_not_adjacent_pairs,num_zeros))}")

            #some stats
            set_num_not_adjacent=0
            set_num_adjacent=0
            current_set_pairs=0
            current_set_not_adj=0
            
            for pair in pairs:
                #if limit of maximum pairs is not exceeded
                if max_elements<0 or num_pairs<=max_elements:
                        
                    num_total_pairs+=1
                    idx1,idx2=pair
                    label=fragment_set["grid"][idx1][idx2]
                    
                    #total number of not adjacent
                    if label==0:
                        set_num_not_adjacent+=1
                    else:
                        set_num_adjacent+=1
                        tot_num_adjacent+=1
                    
                    #if adjacent pair or not adjacent pairs limit not exceeded
                    if label==1 or (label==0 and set_num_not_adjacent<=max_not_adjacent_pairs):

                        #add stats for number of fragments in dataset and current set
                        num_pairs+=1
                        current_set_pairs+=1

                        #add stats for adjacent fragment in dataset
                        if label==0: current_set_not_adj+=1

                        pointcloud1=set_pointcloud[idx1]
                        pointcloud2=set_pointcloud[idx2]

                        #generate pair of pointclouds
                        pointcloud_pair=[ np.asarray(pointcloud1.points) ,  np.asarray(pointcloud2.points) ]

                        normals_pair=[ np.asarray(pointcloud1.normals) ,  np.asarray(pointcloud2.normals) ]

                        #compute where to put this pair
                        destination_set=np.random.choice(["train","test","val"], p=[prob_train, prob_test, prob_val])

                        #check if still available space
                        if destination_set=="train" and num_train_pairs>train_total_pairs:
                            destination_set="test"
                        
                        if destination_set=="test" and num_test_pairs>test_total_pairs:
                            destination_set="val"

                        if destination_set=="val" and num_val_pairs>val_total_pairs:
                            destination_set=""

                        if destination_set:
                            if destination_set=="train":
                                num_train_pairs+=1
                                set_train_data.append(pointcloud_pair)
                                set_train_labels.append(label)
                                set_train_normals.append(normals_pair)

                            elif destination_set=="test":
                                num_test_pairs+=1
                                set_test_data.append(pointcloud_pair)
                                set_test_labels.append(label)
                                set_test_normals.append(normals_pair)
                                
                            elif destination_set=="val":
                                num_val_pairs+=1
                                set_val_data.append(pointcloud_pair)
                                set_val_labels.append(label)
                                set_val_normals.append(normals_pair)


            set_elapsed_time=time.time()-set_start_time
            print("Set completed in: %.3f seconds" % (set_elapsed_time),end=": ")
            print(f"added {current_set_pairs} pairs --> adj: {set_num_adjacent}   n_adj: {current_set_not_adj}")
            print(f"currently dataset contains {num_pairs} pairs",end=". ")
            print()
            
            len_train=len(set_train_data)
            train_data[train_index:train_index+len_train]=set_train_data
            train_labels[train_index:train_index+len_train]=set_train_labels
            train_normals[train_index:train_index+len_train]=set_train_normals
            train_index+=len_train

            len_test=len(set_test_data)
            test_data[test_index:test_index+len_test]=set_test_data
            test_labels[test_index:test_index+len_test]=set_test_labels
            test_normals[test_index:test_index+len_test]=set_test_normals
            test_index+=len_test

            len_val=len(set_val_data)
            val_data[val_index:val_index+len_val]=set_val_data
            val_labels[val_index:val_index+len_val]=set_val_labels
            val_normals[val_index:val_index+len_val]=set_val_normals
            val_index+=len_val



            folder_index+=1


    dataset_file.close()
    total_dataset_time=time.time()-total_dataset_start_time
    print(f"Dataset contains {num_elements} fragments --> {num_total_pairs} total pairs ")
    print("we consider only: ",num_pairs," pairs, of which ", tot_num_adjacent, "are adjacent")
    print(f"Train set: {num_train_pairs},   test set: {num_test_pairs}, val set: {num_val_pairs}")
    print(f"but it should be: train,test,val: {train_total_pairs},{test_total_pairs},{val_total_pairs}")
    print("total time: %.3f seconds" % (total_dataset_time))
    #return np.array(all_data),np.array(all_labels)



#used in class ModelNet40
#for now we don't use it
def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud         



In [9]:
#here i save all the sets of fragments
sets=[]

#root folder
main_path="produzione_29112021"

num_points=100

In [ ]:
count_dataset_pairs(main_path,max_elements=-1,alpha=1)

In [ ]:
create_dataset(main_path,num_points,alpha=1)

Creating dataset...
dataset will contain: 179608  total pairs
file dataset not found, creating
Each point cloud is sampled with 100 points




Starting set 0: folder - generatedTest_2021_11_29_10_00_40
Set stats: 
  --number of fragments: 8
  --total adjacent pairs: 19; total not adjacent pairs: 9
  --dataset for set will contain: 28 pairs
     --> adj: 19    n-adj: 9
Set completed in: 1.295 seconds: added 28 pairs --> adj: 19   n_adj: 9
currently dataset contains 28 pairs. 


Starting set 1: folder - generatedTest_2021_11_29_10_01_59
Set stats: 
  --number of fragments: 8
  --total adjacent pairs: 19; total not adjacent pairs: 9
  --dataset for set will contain: 28 pairs
     --> adj: 19    n-adj: 9
Set completed in: 0.557 seconds: added 28 pairs --> adj: 19   n_adj: 9
currently dataset contains 56 pairs. 


Starting set 2: folder - generatedTest_2021_11_29_10_02_12
Set stats: 
  --number of fragments: 8
  --total adjacent pairs: 18; total not adjacent pairs: 10
  --dataset for set wi